In [14]:
import json, re, time
from pathlib import Path
import ollama

# создание в рабочей директории папки для сохранения результатов
Path("outputs").mkdir(exist_ok=True)
# подготовка файлов для логирования результатов работы программы
trace_path = Path("outputs/yn_trace.jsonl")
result_path = Path("outputs/yn_result.md")
# открыть файл трассировки и записать пустую строку
trace_path.write_text("", encoding="utf-8")  # эффективно очищает файл, если он существовал, или создаёт новый пустой
ts = time.strftime("%Y-%m-%d %H:%M:%S")      # временная метка
# выбор модели
model_name = "llama3.2"
temperature = 0.2            
top_p = 0.9
seed = 8
num_predict = 200
print("Модель:", model_name)



#____ВОПРОС____
question = "Стоит ли использовать искусственный интеллект, если с его помощью можно автоматизировать многие процессы, но он не всегда бывает стабилен?"
print("Вопрос:", question)



#____ПЕРВИЧНАЯ ОЦЕНКА ВОПРОСА ollama____
prompt_A = (
    "Сохранив суть вопроса, преобразуй его в более чёткий вопрос - такой, на который можно ответить да/нет и корректно выдели отдельно допущения и критерии вопроса.\n"
    "Верни строго один JSON-объект без текста до/после, поля:\n"
    "{ \"proposition\": <строка>, \"assumptions\": [строки], \"criteria\": [строки] }.\n"
    "Вопрос:\n\n"
    + question
)

resp_A = ollama.generate(
    model=model_name,
    prompt=prompt_A,
    options={"temperature": temperature, "top_p": top_p, "seed": seed, "num_predict": num_predict}
)
text_A = resp_A["response"].strip()

# распарсить JSON; при неудаче — более строгий повтор
ok_A = False # флаг успешного парсинга
attempt_a = 0  # счётчик попыток
while not ok_A and attempt_a < 3:
    attempt_a += 1
    try:
        data_A = json.loads(text_A)
        ok_A = True
    except Exception as e:
        print("Не JSON, более строгий повтор:...")
        prompt_A_strict = (
            "СТРОГО верни валидный JSON без лишних символов. Формат:\n"
            "{ \"proposition\": <строка>, \"assumptions\": [строки], \"criteria\": [строки] }.\n\nВопрос:\n" + text_A
        )
        resp_A = ollama.generate(
            model=model_name,
            prompt=prompt_A_strict,
            options={"temperature": 0.0, "top_p": 1.0, "seed": seed+attempt_a, "num_predict": 200}
        ) # seed: seed+attempt - изменённый сид для разнообразия
        text_A = resp_A["response"].strip()

# если не получен успешный ответ, используется типизированная заготовка
if not ok_A:
    data_A = {"proposition": question, "assumptions": ["Наиболее упрощённое рассуждение."], "criteria": ["Базовый ответ."]}

print("Оценка вопроса ollama:")
print("\nПропозиция:", data_A.get("proposition",""))
print("Допущения:", data_A.get("assumptions",[]))
print("Критерии:", data_A.get("criteria",[]))

# логирование в файл трассировки
with open(trace_path, "a", encoding="utf-8") as f:
    f.write(json.dumps({"step":"A","ts":ts,"request":prompt_A[:2000],"response":text_A[:2000],"parsed":data_A}, ensure_ascii=False) + "\n")



#____АНАЛИЗ ПРОПОЗИЦИИ ПО РЕЗУЛЬТАТАМ ОЦЕНКИ ollama____
prop, assm, crit = data_A.get("proposition", question), data_A.get("assumptions", []), data_A.get("criteria", [])
context_block = "\n".join(filter(None, [
    f"Пропозиция: {prop}",
    f"Допущения: {'; '.join(assm)}" if assm else "",
    f"Критерии: {'; '.join(crit)}" if crit else ""
]))

prompt_B = (
    "Проанализируй пропозицию да/нет кратко по-русски на 2–4 предложения, ссылаясь на критерии/допущения, "
    "и завершай - VERDICT: YES или VERDICT: NO. Никаких других маркеров после вердикта.\n\n"
    + context_block
)

resp_B = ollama.generate(
    model=model_name,
    prompt=prompt_B,
    options={"temperature": 0.2, "top_p": top_p, "seed": seed, "num_predict": 240}
)
text_B = resp_B["response"].strip()
print(text_B)

with open(trace_path, "a", encoding="utf-8") as f:
    f.write(json.dumps({"step":"B","ts":ts,"request":prompt_B[:2000],"response":text_B[:2000]}, ensure_ascii=False) + "\n")



#____ПРИВЕДЕНИЕ ВЕРДИКТА К БИНАРНОМУ ФОРМАТУ____
binary = None
verdict = None

# поиск VERDICT: YES/NO
lit = re.search(r"VERDICT:\s*(YES|NO)\b", text_B, flags=re.IGNORECASE)
if lit:
    verdict = lit.group(1).upper()

# при неудаче повторный запрос на yes/no
attempt_b = 0
while verdict is None and attempt_b < 3:
    attempt_b += 1
    resp = ollama.generate(
        model=model_name,
        prompt="Ответь строго одним словом на английском: YES или NO. Только это слово. Пропозиция:\n\n" + prop,
        options={"temperature": 0.0, "top_p": 1.0, "seed": seed+attempt_b, "num_predict": 10}
    )
    txt = resp["response"].strip()
    print("Строгий ответ:", txt)
    lit = re.search(r"\b(YES|NO)\b", txt, flags=re.IGNORECASE)
    if lit:
        verdict = lit.group(1).upper()
    with open(trace_path, "a", encoding="utf-8") as f:
        f.write(json.dumps({"step":"B_strict","attempt":attempt_b,"raw":txt}, ensure_ascii=False) + "\n")

# YES/NO в 1/0
if verdict == "YES":
    binary = 1
elif verdict == "NO":
    binary = 0

# при неудаче повторный запрос на 1/0
attempt_c = 0
while binary is None and attempt_c < 3:
    attempt_c += 1
    rule = (
        "Выбери 1 (за) или 0 (против) по пропозиции выше. Если аргументы равны — выбери 0. "
        "Верни строго одну цифру без текста."
    )
    resp = ollama.generate(
        model=model_name,
        prompt=rule + "\n\nПропозиция: " + prop,
        options={"temperature": 0.0, "top_p": 1.0, "seed": seed+10+attempt_c, "num_predict": 5}
    )
    txt = resp["response"].strip()
    print("Строго бинар:", txt)
    lit_2 = re.search(r"^(0|1)$", txt)
    if lit_2:
        binary = int(lit_2.group(1))
    with open(trace_path, "a", encoding="utf-8") as f:
        f.write(json.dumps({"step":"C_force01","attempt":attempt_c,"raw":txt}, ensure_ascii=False) + "\n")

# при неудаче - поиск по всем ответам
if binary is None:
    # YES/NO/ДА/НЕТ из всех ответов
    text_all = (text_A + "\n" + text_B).lower()
    if re.search(r"\b(yes|да)\b", text_all): binary = 1
    if re.search(r"\b(no|нет)\b", text_all): binary = 0 if binary is None else binary

if binary is None:
    # при неудаче - последняя попытка строгой оценки моделью
    resp = ollama.generate(
        model=model_name,
        prompt="Верни строго одну цифру: 1 (за) или 0 (против) по пропозиции ниже. Без комментариев.\n\n" + prop,
        options={"temperature": 0.0, "top_p": 1.0, "seed": seed+99, "num_predict": 5}
    )
    txt = resp["response"].strip()
    print("Финальная попытка:", txt)
    lit_3 = re.search(r"^(0|1)$", txt)
    if lit_3:
        binary = int(lit_3.group(1))

# при неудаче - дефолтное значение 0
if binary is None:
    binary = 0
    print("Не удалось получить бинар от модели, установлено значение 0")

print("\nФИНАЛЬНЫЙ ВЫВОД (0/1):", binary)

Модель: llama3.2
Вопрос: Стоит ли использовать искусственный интеллект, если с его помощью можно автоматизировать многие процессы, но он не всегда бывает стабилен?
Оценка вопроса ollama:

Пропозиция: Использовать искусственный интеллект
Допущения: ['Многие процессы могут быть автоматизированы']
Критерии: ['Стабильность']
Анализ: Пропозиция использования искусственного интеллекта основана на том факте, что многие процессы можно автоматизировать, что приведет к повышению эффективности и стабильности. Однако необходимо учитывать потенциальные риски и ограничения использования ИИ, такие как потеря контроля над процессами или неопределенность в отношении результатов.

Однако, если рассматривать только критерий стабильности, можно заключить, что автоматизация некоторых процессов может привести к повышению стабильности, поскольку ИИ может работать без ошибок и непредсказуемости. 

Следовательно, на основе данных допущений и критериев, я даю следующий ответ:

VERDICT: YES

ФИНАЛЬНЫЙ ВЫВОД (0/1